# Data lineage

{class}`~lamindb.File` objects are the {attr}`~lamindb.Run.inputs` and {attr}`~lamindb.Run.outputs` of run {class}`~lamindb.Run` objects. What is run is a {class}`~lamindb.Transform`.

In each `File` object, access the generating {class}`~lamindb.Run` and {class}`~lamindb.Transform` objects via {attr}`lamindb.File.run` and {attr}`lamindb.File.transform`.

In [ ]:
import lamindb as ln

## Notebooks

A global run context is created upon `ln.track()`:

In [ ]:
ln.track()

In [ ]:
ln.context.run

Let's query from which notebook file "iris_new" was ingested:

In [ ]:
with ln.Session() as ss:
    file = ss.select(ln.File, name="iris_new").one()
    print(file.run.transform)

## Pipelines

In [ ]:
filepath = ln.dev.datasets.file_fastq()

When working with a pipeline, we'll register it before running it.

In [ ]:
transform = ln.Transform(name="10x scRNA-seq nextseq", type="pipeline")

We can then use the {func}`~lamindb.track` as before:

In [ ]:
ln.track(transform)

In [ ]:
file_fastq = ln.File(filepath)

In [ ]:
ln.add(file_fastq)

:::{dropdown}

We can also manually pass a run and not use the global run context set by `ln.track`:
```
run = ln.Run(transform=transform, name="ingest-fastq")
ln.File(filepath, run=run)
```

:::

## Track run inputs

While run outputs are automatically tracked as data sources, run inputs aren't.

Let's register a pipeline that takes the fastq file as an input:

In [ ]:
ln.track(ln.Transform(name="Cell Ranger", version="7", type="pipeline"))

To track it as an input for the current run, set `is_run_input=True`.

In [ ]:
file_fastq = ln.select(ln.File, name="input.fastq.gz").one()
file_fastq.stage(is_run_input=True)

Let's get an exemplary output filepath:

In [ ]:
output_filepath = ln.dev.datasets.file_bam()

In [ ]:
output_filepath

In [ ]:
file = ln.File(output_filepath)

ln.add(file)

Let's query the input file of the run that produced file `output.bam`:

In [ ]:
with ln.Session() as ss:
    file = ss.select(ln.File, name="output.bam").one()
    print(file.run.inputs)

In [ ]:
assert file.run.inputs[0].name == "input.fastq.gz"